# Api Calling

In [2]:
import pandas as pd
from opencage.geocoder import OpenCageGeocode

def get_location_details(latitude, longitude, key):
    """Fetch location details using OpenCage reverse geocode API."""
    geocoder = OpenCageGeocode(key)
    try:
        results = geocoder.reverse_geocode(latitude, longitude)
        if results and len(results) > 0:
            return results[0]['formatted']  
        else:
            return "No location found"
    except Exception as e:
        return f"Error fetching data: {str(e)}"  

def enrich_airbnb_data(data_path, key, num_rows):
    
    """Enrich Airbnb data with detailed location information for a specified number of rows."""
    # Load the dataset
    data = pd.read_csv(data_path)

    # Process only the first 'num_rows' entries
    subset_data = data.head(num_rows)

    # Apply reverse geocoding to each row in the subset
    subset_data['detailed_location'] = subset_data.apply(
        lambda row: get_location_details(row['latitude'], row['longitude'], key), axis=1
    )

    # Save the updated DataFrame to a new CSV file
    subset_data.to_csv('updated_airbnb_data_with_locations_subset.csv', index=False)
    
    # Return the subset DataFrame for verification or further use
    return subset_data

# Set the path to your dataset and your OpenCage API key
data_path = 'https://raw.githubusercontent.com/deepakguda01/IA_Final_Project/main/NYC-Airbnb-2023%20.csv'
key = 'API Key' 
num_rows = 500 

# Enrich the dataset
enriched_data_subset = enrich_airbnb_data(data_path, key, num_rows)

# Print the first few rows of the enriched data to check
enriched_data_subset



C:\Users\deepa\AppData\Local\Temp\ipykernel_8560\1213734621.py:20: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)
C:\Users\deepa\AppData\Local\Temp\ipykernel_8560\1213734621.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['detailed_location'] = subset_data.apply(


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,detailed_location
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,49,2022-06-21,0.30,3,314,1,NaN,"104 West 40th Street, New York, NY 10018, Unit..."
1,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.30,2,365,0,NaN,"294 Gates Avenue, New York, NY 11216, United S..."
2,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80380,-73.96751,Private room,75,2,118,2017-07-21,0.72,1,0,0,NaN,"301 West 109th Street, New York, NY 10025, Uni..."
3,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,575,2023-02-19,3.41,1,106,52,NaN,"Hilton Garden Inn, 237 West 54th Street, New Y..."
4,5136,"Large Sunny Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,60,3,2022-08-10,0.03,1,181,1,NaN,"Al-Noor School, 675 4th Avenue, New York, NY 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,470498,*Unique Master BR in Battery Park!*,2334269,Filip,Manhattan,Battery Park City,40.71051,-74.01724,Private room,65,30,8,2015-12-22,0.08,1,0,0,NaN,"Albany Street, New York, NY 10006, United Stat..."
496,328040,"Awesome views, Central location",1676487,Robyn,Manhattan,Murray Hill,40.74838,-73.98214,Entire home/apt,349,5,36,2023-01-06,0.31,1,83,3,NaN,"Church of the Incarnation, 209 Madison Avenue,..."
497,396636,Stylish Designer Studio with Piano,1981742,Daniel,Manhattan,Harlem,40.82903,-73.94204,Entire home/apt,135,30,42,2019-12-10,0.32,1,277,0,NaN,"848 Saint Nicholas Avenue, New York, NY 10031,..."
498,471712,Beautiful Sunny Apartment in South Harlem,1274269,Aram,Manhattan,Harlem,40.80631,-73.95579,Private room,86,30,74,2019-11-26,0.57,1,0,0,NaN,"322 West 118th Street, New York, NY 10026, Uni..."


# Bucket Creation

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region. If a region is not specified,
    the bucket is created in the S3 default region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-east-2'
    :return: True if bucket created, else False
    """
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name, CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

# Attempt to create the bucket
bucket_name = 'group-1-final-project-sourcesdata'  
region = 'us-east-2'  

if create_bucket(bucket_name, region):
    print(f'Bucket "{bucket_name}" created successfully.')
else:
    print(f'Failed to create bucket "{bucket_name}". Please check the logs for more details.')


In [ ]:
# Attempt to create the bucket
bucket_name = 'group-1-final-project-finalsdata'
region = 'us-east-2' 

if create_bucket(bucket_name, region):
    print(f'Bucket "{bucket_name}" created successfully.')
else:
    print(f'Failed to create bucket "{bucket_name}". bucket already exit ')

# Uploading Downloaded csv into bucket

In [ ]:
import boto3

def upload_csv_to_s3(file_name, bucket_name, object_name=None):

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket_name, object_name)
    except Exception as e:
        print(f"Upload failed: {e}")
        return "not uploaded"
    return 'uploaded'

bucket_name =  'group-1-final-project-sourcesdata'
csv_file_name = 'NYC_Airbnb.csv'
api_file_name = 'updated_airbnb_data_with_locations_subset.csv'


upload_csv_to_s3(csv_file_name, bucket_name)
upload_csv_to_s3(api_file_name, bucket_name)